In [1]:
import warnings
warnings.filterwarnings("ignore")
from langchain.text_splitter import RecursiveCharacterTextSplitter
from newspaper import Article
from sentence_transformers import SentenceTransformer
import faiss
import os
import re
import google.generativeai as genai
from security import MY_API_KEY

In [32]:
class quesitionChatBot:

    def __init__(self , chunk_size=1000):

        genai.configure(api_key = MY_API_KEY)

        self.model = genai.GenerativeModel(model_name = "gemini-1.5-flash")
        self.encoder = SentenceTransformer("all-mpnet-base-v2")
        self.splitter = RecursiveCharacterTextSplitter(chunk_size =chunk_size , chunk_overlap = 200)
        self.index = faiss.IndexFlatL2()
        self.vectors = []
        self.texts = ""
        self.chunks = []
        self.error = True
        self.url = ""
        #self.default_prompts = [ "Where the place is located. " , "How to book and procedure to book." ,  "what are the special offer and discount" , 
         #                       "How to contact , phone or email address"]

        self.prompt = "Use the following portion of a long document to see if any of the text    is proper the question. \nReturn relevent text varbit.\n"




    def getTextFromURLS(self,URL):

        self.texts = ""

        try:

            article = Article(URL)
            article.download()
            article.parse()
            self.texts = article.text[:3000]
            
        except:
            return  "The  Entered URL is invalid or permission is not granted format [ URL:<your_url> ]"
        
        self.error = False
        self.url = URL
        self.textToChunks()
        return f"{self.url} URL is scanned successfully...!"
    
    
    def textToChunks(self):


        self.chunks = self.splitter.split_text(self.texts)
        
        self.chunksToVectorsAndIndex()

        print(len(self.chunks))
        return len(self.chunks)
    
    def chunksToVectorsAndIndex(self):

        self.vectors = self.encoder.encode(self.chunks)
        self.index = faiss.IndexFlatL2( self.vectors.shape[1])
        self.index.add(self.vectors)
    
    def getResut(self,question):

        if("url:" in question.lower()):

            index = re.search("http\S*",question)

            if(index == None):
                return "Enter URL in correct format. format [ URL:<your_url> ]"
            
            index = index.span()
            url = question[index[0] : index[1]]
        
            return self.getTextFromURLS(url)
        
        elif( "review:" in question.lower()):

            return "Your review is verified successfully"
        
        return self.getAnswerFromGAI(question)
        
    
    def getAnswerFromGAI(self , question , k=5):

        if(self.error):
            return "Please Enter valid URL format [ URL:<your_url> ]"

        vec = self.encoder.encode([question])
        _ , idx = self.index.search(vec , k = k)

        resopnces = ""

        for i in idx[0]:
            resopnces+= "\n" +( self.model.generate_content( f"{self.prompt + self.chunks[i] }\nquestion: {question}\n Relevent text if any: ").text)
        
        final_output = self.model.generate_content( f" {self.prompt}\n{resopnces}\n question: {question}\n Relevent text if any: ").text + f"\n\n\n[Results are dispalyes from {self.url}]"

        
        return  final_output

In [1]:
import pandas as pd

In [4]:
s = pd.read_csv("processed_hotels.csv")

C:\Users\mohan\AppData\Local\Temp\ipykernel_29536\146805155.py:1: DtypeWarning: Columns (20,22,24,27,29,30,32,34,36,38,39,42,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  s = pd.read_csv("processed_hotels.csv")


In [7]:
s["City"]

0        Santa  Barbara
1                Affton
2          Philadelphia
3            Green Lane
4          Ashland City
              ...      
96988            Tucson
96989      Philadelphia
96990          Voorhees
96991      Indianapolis
96992             Aston
Name: City, Length: 96993, dtype: object

In [9]:
a = """{'Start': 'chennai', 'Destination': 'AVON', 'modeOfTransport': 'flight', 'num_days': 5}
{
    "day-1": {
        "1" : "fly from chennai to avon (flight details will be needed to calculate flight time and potential layovers)",
        "2" : "check into hotel in avon",
        "3" : "explore the local area and find a place for dinner",
        "5" : "enjoy a relaxed evening"
    },
    "day-2": {
        "1" : "visit [attraction 1 in avon]",
        "2" : "have lunch at a local restaurant",
        "3" : "visit [attraction 2 in avon]",
        "5" : "dinner at a restaurant of your choice"
    },
    "day-3": {
        "1" : "visit [attraction 3 in avon]",
        "2" : "enjoy a picnic lunch in a park",
        "3" : "visit [attraction 4 in avon]",
        "5" : "dinner at a restaurant of your choice"
    },
    "day-4": {
        "1" : "visit [attraction 5 in avon]",
        "2" : "have lunch at a restaurant",
        "3" : "visit [attraction 6 in avon]",
        "5" : "dinner at a restaurant of your choice"
    },
    "day-5": {
        "1" : "enjoy breakfast at your hotel",
        "2" : "pack your bags and check out",
        "3" : "fly back to chennai"
    },
    "cost" : "unknown",
    "estimation" :[ "to provide an accurate cost for your trip, i need more information about specific flight details, hotel choices, and activities you plan to do in avon.  flight costs can vary greatly depending on the airline, time of year, and availability. you can use flight search engines like kayak, google flights, or skyscanner to compare prices for flights from chennai to avon. remember to factor in costs for meals, transportation, and any entrance fees for attractions.",
                 "hotel prices can also vary depending on location, amenities, and time of year. you can use websites like booking.com, expedia, or hotels.com to find and compare hotel options. when making your choice, consider the distance from the hotel to attractions and transportation options.",
                 "once you have a clearer idea of your travel dates, flight costs, and hotel options, you can estimate the total cost of your trip.",
                 "please note, that avon, massachusetts, is a small town. there may not be direct flights from chennai, so you might need to factor in connecting flights, which would increase the overall cost of your trip."]
    }"""

In [13]:
st = re.search("(``)*json",a.lower())
st

In [11]:
import re

In [48]:
a = """{
    "Day-1": {
        "1" : "Fly from King of Prussia to Madurai (via a connecting flight). You could find flights from Philadelphia International Airport (PHL) to Madurai International Airport (IXM).",
        "2" : "Arrive at Madurai Airport and transfer to your hotel.",
        "3" : "Check in and freshen up. ",
        "4" : "Explore the Meenakshi Amman Temple, a magnificent Hindu temple known for its colorful gopurams (temple towers)."
        },
    "Day-2": {
        "1" : "Visit the Thirumalai Nayakkar Palace, a grand palace built in the 17th century.",
        "2" : "Explore the Gandhi Museum, which showcases the life and works of Mahatma Gandhi.",
        "3" : "Enjoy a traditional South Indian lunch at a local restaurant."
        },
    "Day-3": {
        "1" : "Take a day trip to Kodaikanal, a hill station known for its picturesque landscapes.",
        "2" : "Explore the Kodaikanal Lake, Coaker's Walk, and the Bryant Park.",
        "3" : "Enjoy a scenic drive back to Madurai."
        },
    "Day-4": {
        "1" : "Visit the Alagar Koyil Temple, a historic temple dedicated to Lord Vishnu."
        "2" : "Enjoy some shopping at local markets for souvenirs and handicrafts."
        "3" : "Have dinner at one of Madurai's famous restaurants."
        "4" : "Transfer to Madurai Airport for your flight back."
        },
    "Cost" : "800$" ,
    "Estimation" :[
        "Flight tickets from King of Prussia to Madurai: $500-$700 (round trip, depending on the time of year and availability).",
        "Hotel accommodation: $100-$150 per night.",
        "Local transportation: $50-$100 (including taxis, buses, and auto-rickshaws).",
        "Food and drinks: $50-$100 (depending on your preferences).",
        "Entrance fees to attractions: $20-$50 (depending on the attractions you visit).",
        "Miscellaneous expenses: $20-$50"
    ]
    }"""

a = a.lower()

In [55]:
b = re.search("\{\s*\"day\s*-*1",a)
b

<re.Match object; span=(0, 12), match='{\n    "day-1'>

In [56]:
a = dict()
a== "hii"

False

In [43]:
re.search("{\s*\"d",'  {"d}')